In [7]:
#использую реализацию Gensim Doc2Vec
#import re  # Для препроцессинга
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [8]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [9]:
#токенизация и добавление в текст
description = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df_clean.clean)]

In [10]:
#размер вектора, количество измерений, количество переборов, количество ядер
d2v_model = Doc2Vec(vector_size=64, min_count=1, epochs = 20, workers=cores)

INFO - 01:32:19: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w5,s0.001,t12)', 'datetime': '2022-11-19T01:32:19.011810', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [11]:
#Модель
d2v_model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=cores, epochs = 40)
#Словарь
d2v_model.build_vocab(description)
#Обучение модели
d2v_model.train(description, total_examples=d2v_model.corpus_count
            , epochs=d2v_model.epochs)

INFO - 01:32:21: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w2,s0.001,t12)', 'datetime': '2022-11-19T01:32:21.693577', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 01:32:21: collecting all words and their counts
INFO - 01:32:21: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 01:32:21: PROGRESS: at example #10000, processed 624502 words (3888712/s), 14928 word types, 0 tags
INFO - 01:32:22: PROGRESS: at example #20000, processed 1190758 words (3751674/s), 20609 word types, 0 tags
INFO - 01:32:22: PROGRESS: at example #30000, processed 1739093 words (3441388/s), 24271 word types, 0 tags
INFO - 01:32:22: PROGRESS: at example #40000, processed 2877387 words (3690211/s), 30723 word types, 0 tags
INFO - 01:32:22: PROGRESS: at example #50000, processed 3899786 words (3488701/s), 36226 word types, 0 tags
INFO - 01:32:23: PROGRESS: a

In [13]:
#Генерация векторов
des2vec = [d2v_model.infer_vector((df_clean['clean'][i].split(' '))) 
            for i in range(0,len(df_clean['clean']))]
des2vec

[array([-0.11078861,  0.38233045, -0.02895567,  0.66740906,  0.7815356 ,
        -0.7040208 ,  0.53246725, -0.23855138,  0.23816486, -0.24592184,
         0.36282206,  1.165151  , -0.19796868, -0.14203851,  0.25972497,
         0.33708525,  0.09891965, -0.54958016, -0.17224807,  0.21599238,
         0.76671785,  0.45605287,  0.36856198,  0.07264187, -0.13133264,
         0.4119866 ,  0.19830738, -0.44288433,  0.17348967, -0.21259983,
        -0.8255987 , -0.48915935,  0.3876178 ,  0.42571372, -0.24868715,
         0.9847862 , -0.1885647 , -0.1629265 , -0.08375676, -0.5984896 ,
        -0.47445628, -0.32281178, -0.6592606 ,  0.2388825 , -0.28492954,
         0.98764896,  0.61749065, -0.39941642, -0.25749445,  0.2663358 ,
        -0.03245178,  0.76799923,  0.3645677 , -0.09978083,  0.6122048 ,
        -0.31900972,  0.03960925, -0.0729267 , -0.17357865,  0.40163952,
        -0.18293855,  0.19873019,  0.1824062 ,  1.1963755 ], dtype=float32),
 array([ 0.63216895, -0.0240716 ,  0.3139016 , 

In [14]:
import numpy as np
#Создание списка списков
#добавление списка в столбец данных
df_clean['doc2vec'] = np.array(des2vec).tolist()
df_clean.head(2)

,Unnamed: 0,clean,doc2vec
0,0,прекрасный однокомнатный квартира дом лифт мус...,"[-0.11078860610723495, 0.38233044743537903, -0..."
1,1,светлый просторный комната кв метр отдельный л...,"[0.6321689486503601, -0.024071604013442993, 0...."


In [15]:
df_clean = df_clean.drop(['Unnamed: 0'],axis=1)

In [16]:
df_clean['id'] = df_clean.index
df_clean.head(2)

,clean,doc2vec,id
0,прекрасный однокомнатный квартира дом лифт мус...,"[-0.11078860610723495, 0.38233044743537903, -0...",0
1,светлый просторный комната кв метр отдельный л...,"[0.6321689486503601, -0.024071604013442993, 0....",1


In [17]:
df_clean.to_csv('Doc2Vec.csv',index=False)

In [18]:
df_clean['doc2vec']

0        [-0.11078860610723495, 0.38233044743537903, -0...
1        [0.6321689486503601, -0.024071604013442993, 0....
2        [0.5211401581764221, 0.22238247096538544, 0.40...
3        [0.7995444536209106, 0.32308828830718994, -0.1...
4        [0.2922152578830719, 0.7411530613899231, -0.06...
                               ...                        
65108    [0.32550597190856934, -0.19205866754055023, -0...
65109    [0.5037409067153931, -0.359331339597702, 0.343...
65110    [0.19771675765514374, -0.037163421511650085, -...
65111    [0.376088410615921, 0.2511630952358246, 0.3691...
65112    [0.08633612841367722, -0.21015778183937073, -0...
Name: doc2vec, Length: 65113, dtype: object